In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score,classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score


In [3]:
df = pd.read_csv('/content/drive/My Drive/project1/df_test2.csv',index_col=[0]) 

In [4]:
df

,text,Open,High,Low,Close,VIX,AAL,AAPL,AMD,AMZN,CCL,GOOG,GOOGL,INTC,JNJ,NFLX,NVDA,TSLA,TWTR,ROC_prediction_label
Date,,,,,,,,,,,,,,,,,,,,
2017-01-04,target stock target stock leader self driving ...,0.008061,0.009671,0.009050,0.011383,11.850000,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2017-01-04,everyone confident target stock amd really tra...,0.012558,0.013607,0.010804,0.014647,11.850000,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2017-01-04,everyone confident nvda target stock target st...,0.026704,0.019566,0.025109,0.021660,11.850000,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2017-01-04,target stock interesting watch week think quar...,0.000000,0.000000,0.000000,0.000503,11.850000,0,0,0,0,0,0,0,0,0,1,0,0,0,2
2017-01-05,might sell tsla got looking last years data tr...,0.010851,0.009554,0.010906,0.011324,11.670000,0,0,0,0,0,0,0,0,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-03,bot identified tracked following options picks...,0.339861,0.332454,0.338624,0.322640,18.040001,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-06-03,clne target stock jets nclh mains buy target s...,0.391106,0.384888,0.381815,0.384681,18.040001,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2021-06-03,tsla announced dis target stock collaborative ...,0.836041,0.831019,0.845417,0.836135,18.040001,0,1,0,0,0,0,0,0,0,0,0,0,0,2


In [5]:
def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1))

In [6]:
vectorized = vectorizer.fit_transform(df['text'])

In [7]:
vectorized = pd.DataFrame(vectorized.todense(), columns=vectorizer.get_feature_names())

In [8]:
df.index

Index(['2017-01-04', '2017-01-04', '2017-01-04', '2017-01-04', '2017-01-05',
       '2017-01-05', '2017-01-05', '2017-01-05', '2017-01-06', '2017-01-06',
       ...
       '2021-06-02', '2021-06-02', '2021-06-03', '2021-06-03', '2021-06-03',
       '2021-06-03', '2021-06-03', '2021-06-03', '2021-06-03', '2021-06-03'],
      dtype='object', name='Date', length=9184)

In [9]:
vectorized.index = df.index

In [10]:
df = df.drop(columns=['text'])

In [11]:
df = pd.concat([df,vectorized],axis = 1)

In [12]:
del vectorized 

In [13]:
df.head()

,Open,High,Low,Close,VIX,AAL,AAPL,AMD,AMZN,CCL,GOOG,GOOGL,INTC,JNJ,NFLX,NVDA,TSLA,TWTR,ROC_prediction_label,aaa,aaaaaaaaaaaaaa,aaaaaaaah,aaaaaaaand,aaaaaaahahahahaha,aaaaaaand,aaaaaahh,aaaaaand,aaaaah,aaaaand,aaaaannny,aaaahhhahahaha,aaaallllll,aaaand,aaaattttt,aaaayyyyyy,aaand,aaany,aaapl,aaayyy,aab,...,zuf,zuh,zuhfhoj,zuk,zumiez,zune,zuo,zurueck,zuvcm,zvjwye,zvo,zvquldn,zwc,zwehq,zwp,zwvriiwic,zwvriiwidgltzvvuaxqiom,zwxe,zxi,zxj,zxjab,zxjdagfydci,zxjdb,zxjzijp,zxmurr,zxya,zyn,zyne,zynga,zynq,zyvrj,zzhn,zzjzn,zzxjpzxmilcjzew,zzz,zzzz,zzzzz,zzzzzz,zzzzzzz,zzzzzzzzzzzzzz
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-04,0.008061,0.009671,0.009050,0.011383,11.85,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-04,0.012558,0.013607,0.010804,0.014647,11.85,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-04,0.026704,0.019566,0.025109,0.021660,11.85,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-04,0.000000,0.000000,0.000000,0.000503,11.85,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-05,0.010851,0.009554,0.010906,0.011324,11.67,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
labels = df['ROC_prediction_label']
input = df.drop(columns=['ROC_prediction_label'])

In [15]:
del df

In [16]:
# clf = SVC(gamma='auto')
# clf.fit(input, labels)

In [19]:
train1_inp, val1_inp = input[:6000], input[6000:7000]
train1_label,val1_label = labels[:6000], labels[6000:7000]

train2_inp, val2_inp = input[:7000], input[7000:8000]
train2_label,val2_label = labels[:7000], labels[7000:8000]

train3_inp, val3_inp = input[:8000], input[8000:9000]
train3_label,val3_label = labels[:8000], labels[8000:9000]

In [20]:
clf = SVC(gamma='auto')
clf.fit(train1_inp, train1_label)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [31]:
clf2 = SVC(gamma='auto')
clf2.fit(train2_inp, train2_label)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
clf3 = SVC(gamma='auto')
clf3.fit(train3_inp, train3_label)

In [33]:
#pred_proba = clf.predict_proba(val1_inp)[:, 1]
pred = clf2.predict(val2_inp)        

In [30]:
acc = accuracy_score(val1_label, pred)
print(classification_report(val1_label, pred))

              precision    recall  f1-score   support

           0       0.46      0.21      0.29       371
           1       0.00      0.00      0.00       164
           2       0.47      0.85      0.61       465

    accuracy                           0.47      1000
   macro avg       0.31      0.35      0.30      1000
weighted avg       0.39      0.47      0.39      1000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
